# **This notebook is about finding similarity between two titles of the product using word2vec and cosine similarity. This feature can be used with image to find the similar product.**

In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from gensim.models import Word2Vec
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('english')

from numpy import dot
from numpy.linalg import norm
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package wordnet to /Users/sibish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [41]:
train_df = pd.read_csv('train.csv')
#test_df = pd.read_csv('../input/shopee-product-matching/test.csv')
#DATA_PATH = '../input/shopee-product-matching/'

# Add target column in the dataframe

In [42]:
#train_df['image'] = DATA_PATH + 'train_images/' + train_df['image']
tmp = train_df.groupby('label_group').posting_id.agg('unique').to_dict()
train_df['target'] = train_df.label_group.map(tmp)
train_df

,posting_id,image,image_phash,title,label_group,target
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]"
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",2937985045,"[train_3386243561, train_3423213080]"
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]"
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,4093212188,"[train_2406599165, train_3342059966]"
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]"
...,...,...,...,...,...,...
34245,train_4028265689,fff1c07ceefc2c970a7964cfb81981c5.jpg,e3cd72389f248f21,Masker Bahan Kain Spunbond Non Woven 75 gsm 3 ply lapis Bisa Dicuci,3776555725,"[train_2829161572, train_4028265689]"
34246,train_769054909,fff401691371bdcb382a0d9075dfea6a.jpg,be86851f72e2853c,MamyPoko Pants Royal Soft - S 70 - Popok Celana,2736479533,"[train_1463059254, train_769054909]"
34247,train_614977732,fff421b78fa7284284724baf249f522e.jpg,ad27f0d08c0fcbf0,KHANZAACC Robot RE101S 1.2mm Subwoofer Bass Metal Wired Headset,4101248785,"[train_4126022211, train_3926241003, train_2325457554, train_9568348, train_512157627, train_1264798465, train_603410791, train_1369506345, train_614977732]"
34248,train_3630949769,fff51b87916dbfb6d0f8faa01bee67b8.jpg,e3b13bd1d896c05c,"Kaldu NON MSG HALAL Mama Kamu Ayam Kampung , Sapi Lokal, Jamur (Bkn Alsultan / Biocell)",1663538013,"[train_3419392575, train_1431563868, train_3630949769]"


In [43]:
list = train_df[train_df['posting_id'] == 'train_129225211']['target'][0]
list = ' '.join(list)
list

'train_129225211 train_2278313361'

In [44]:
def target_string(list):
    return ' '.join(list)
    
train_df['target_string'] = train_df.apply(lambda x: target_string(x['target']),axis=1)

In [45]:
train_df.head(2)

,posting_id,image,image_phash,title,label_group,target,target_string
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]",train_129225211 train_2278313361
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",2937985045,"[train_3386243561, train_3423213080]",train_3386243561 train_3423213080


In [46]:
train_df[train_df['posting_id'] == 'train_2278313361']

,posting_id,image,image_phash,title,label_group,target,target_string
33161,train_2278313361,f83b49a86a0ee8592e3bf0204da3fbdf.jpg,ac63931c3d4b42f6,PAPER BAG VICTORIA SECRET,249114794,"[train_129225211, train_2278313361]",train_129225211 train_2278313361


In [47]:
train_df['title'][5]


'CELANA WANITA  (BB 45-84 KG)Harem wanita (bisa cod)'

# Data Cleaning

In [48]:

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            if token == 'xxxx':
                continue
            result.append(lemmatize_stemming(token))
    
    return result

In [49]:
processed_docs = train_df['title'].map(preprocess)
#processed_docs =list(processed_docs)


In [50]:
processed_docs[:10] # clean document

0                                                                                       [paper, victoria, secret]
1                                                                        [doubl, tape, origin, doubl, foam, tape]
2                                                                               [male, can, pork, luncheon, meat]
3                        [daster, batik, lengan, pendek, motif, acak, campur, leher, kanc, batik, karakt, alhadi]
4                                                                                           [nescaf, clair, latt]
5                                                                           [celana, wanita, harem, wanita, bisa]
6                                                                                             [jubah, anak, size]
7            [kulot, plisket, salur, candi, plisket, wish, kulot, premium, kulot, pelangi, premium, hieka, kulot]
8                                                 [logu, tempelan, kulka, magnet, angka,

# Word2vec model

I choose embedding dim of size 50. This means that each word will be represented by a vector of size 50

In [51]:
def word2vec_model():
    w2v_model = Word2Vec(min_count=1,
                     window=3,
                     vector_size=50,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)
    
    w2v_model.build_vocab(processed_docs)
    w2v_model.train(processed_docs, total_examples=w2v_model.corpus_count, epochs=300, report_delay=1)
    
    return w2v_model

In [52]:
w2v_model = word2vec_model()
w2v_model.save('word2vec_model')

# Getting embedding vector

In [53]:
emb_vec = w2v_model.wv

In [54]:
emb_vec['anak'] # It will return vector representation of the word anak

array([ 0.63164914, -0.05297348,  0.5677213 ,  1.3718855 , -1.1844982 ,
        0.6978742 ,  0.18490711, -1.0322958 ,  0.19902295, -0.4442447 ,
       -0.03811399,  1.1039885 ,  0.19344488, -1.1717407 , -0.22867925,
        0.4608247 , -0.10700428,  0.30817065, -1.0160388 , -0.37560576,
       -0.11114025, -1.9338365 ,  0.44289273, -0.14635788,  0.9816865 ,
        1.1857678 , -0.28073683,  0.98689884, -1.4834789 , -0.69989073,
       -0.20969453, -0.6106951 ,  0.56077033, -2.2924953 , -1.1198221 ,
        1.6667485 , -0.37312406,  0.03206047, -0.19668548,  0.50741845,
       -0.3260784 ,  0.5314156 ,  2.636372  , -1.1646401 ,  0.9520374 ,
       -1.4113377 , -0.32053667,  1.44694   ,  0.0678439 ,  1.4429355 ],
      dtype=float32)

# Finding similarity between two vector using cosine similarity

In [55]:

def find_similarity(sen1, sen2, model):
    p_sen1 = preprocess(sen1)
    p_sen2 = preprocess(sen2)
    
    sen_vec1 = np.zeros(50)
    sen_vec2 = np.zeros(50)
    for val in p_sen1:
        sen_vec1 = np.add(sen_vec1, model[val])

    for val in p_sen2:
        sen_vec2 = np.add(sen_vec2, model[val])
    
    return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))
    

In [56]:
find_similarity('Atasan Rajut Wanita LISDIA SWEATER', 'Atasan Rajut Wanita LISDIA',emb_vec )

0.9796459529157003

In [57]:
find_similarity('Atasan Rajut Wanita LISDIA SWEATER', 'CELANA WANITA  (BB 45-84 KG)Harem wanita (bisa cod)',emb_vec )

0.6992784224995383

# Fell free to use this notebook and please upvote if you like the work.
# Thank You

In [58]:
#submission_df = pd.DataFrame()
#for match_index, match_row in test_df.iterrows():
#    print(match_row['title'])
#    match_title=match_row['title']
#    matched = ""
#    for index, row in test_df.iterrows():
#        current_title=row['title']
#        #print('match_title ', match_title)
#        #print('current_title ', current_title)
#        sim = find_similarity(match_title, current_title,emb_vec )
#        #print(sim)
#        if sim > 0.9:
#            if len(matched)< 1:
#                matched = row['posting_id']
#            else:
#                matched = matched + " " + row['posting_id']
#    print("matched ", matched)    
#    submission_df = submission_df.append({'posting_id' : match_row['posting_id'] , 'matches' : matched}, 
#               ignore_index = True)

In [59]:
#submission_df = submission_df[["posting_id", "matches"]]
#submission_df

In [60]:
#submission_df.to_csv('submission.csv', index=False)

In [61]:
#f = open("submission.csv", "r")
#print(f.read())

In [62]:
#f = open("../input/shopee-product-matching/sample_submission.csv", "r")
#print(f.read())

In [63]:
train_df_val = train_df.head(1000)

In [64]:
submission_df_val = pd.DataFrame()
for match_index, match_row in train_df_val.iterrows():
    print(match_row['title'])
    match_title=match_row['title']
    matched = ""
    for index, row in train_df.iterrows():
        current_title=row['title']
        sim = find_similarity(match_title, current_title,emb_vec )
        if sim > 0.9:
            if len(matched)< 1:
                matched = row['posting_id']
            else:
                matched = matched + " " + row['posting_id']
    print("matched ", matched)    
    #train_df_val.loc[match_index]['predicted'] = matched
    train_df_val.loc[match_index,'predicted'] = matched
    

    #submission_df_val = submission_df_val.append({'posting_id' : match_row['posting_id'] , 'matches' : matched}, 
    #            ignore_index = True)

Paper Bag Victoria Secret


<ipython-input-55-f0148efcbc74>:13: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


matched  train_129225211 train_2278313361
Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE


/Users/sibish/opt/miniconda3/envs/cs7643-a2/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/sibish/opt/miniconda3/envs/cs7643-a2/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


matched  train_3386243561 train_860027362 train_211200223 train_1974987067
Maling TTS Canned Pork Luncheon Meat 397 gr
matched  train_2288590299 train_3803689425
Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi
matched  train_2406599165 train_3576714541 train_1508100548 train_466135735 train_2043094887 train_1744956981 train_3526771004
Nescafe \xc3\x89clair Latte 220ml
matched  train_3369186413 train_921438619
CELANA WANITA  (BB 45-84 KG)Harem wanita (bisa cod)
matched  train_2464356923 train_4016842252 train_3717368125 train_605380628 train_3827494313 train_1759676686 train_956165758 train_818562223 train_718919006 train_410033747 train_3802582694 train_4039731280 train_742888211 train_3662993631 train_1915376122 train_997480032 train_718380210 train_413879832 train_3002955302 train_3395454388 train_2600816306 train_2312840791 train_3648644051 train_1337416323 train_906952721 train_1007599040 train_2742414112 train_3306758054 train_311

...

matched  train_3330762048
Rak Sepatu 5th Rangka Besi & Plastik (Lemari Sepatu 5 Tingkat)
matched  train_2853268808 train_1403810126 train_2308430657 train_3015487282 train_4158394028 train_2811043292 train_4239345453 train_1093272676 train_1401990065 train_818202932 train_4019835600 train_3732825004 train_393866642
Masker Al Fatih by Rbc Dokter Ressti BPOM ORI 1000 persen n termurah
matched  train_1312941666
Rasha_Sepatu adidas alphabounce terbaru size 36-44.. Bisa cod..!!!
matched  train_1759919213
Berrybenka - Masker Kain Benka Reusable Cup Mask
matched  train_1415324675 train_1478000026
PREMIUM Full Cover Tempered Glass 5D 6D 9D Warna Xiaomi Redmi 5 Plus (5.99 inch)
matched  train_3763177524 train_3058896784 train_1525618070 train_951578623 train_3458937209 train_647662475 train_3507499616 train_3030631503 train_2904440967 train_1648344576 train_2689063600 train_3928134180 train_4265417650 train_1642491796 train_3769291714 train_1737418010 train_3159243756 train_1710687997 train_219

matched  train_3809991483 train_1660164655 train_4193006014 train_540840396 train_3803549515
NEW WAISTBAG ALA KOREA - Tas Selempang Pria Wanita
matched  train_1232041720 train_2869779660 train_1065906242 train_4141908750 train_139346333 train_2605502967 train_2247780299 train_801521840 train_1652717296 train_3395309503 train_219315523 train_4181834509 train_1702727283 train_3063680560 train_1298274482 train_420396090 train_1551895204 train_468451127 train_426994012 train_3753570429 train_3046196729 train_3158987103 train_744421882 train_4207086980 train_3807785384 train_3541138696 train_2140425851 train_683018646 train_1043145560 train_3775712455 train_3539495273 train_2802555132 train_839939872 train_2118909616 train_1599197154 train_2200953573 train_200903631 train_3687072759 train_820534447 train_1711572270 train_3530558534 train_958153727 train_1647381843 train_215301090 train_753745143 train_1855500925 train_3958817816 train_627328419 train_2166949404 train_3647458940 train_277592

matched  train_2559532669 train_3599739122 train_197769535 train_3181829535 train_2251847720 train_1634182663 train_3750944033 train_1379503025 train_1868076887 train_2752327336 train_3913168931 train_1294487234 train_2925437527 train_3132135789
MITU Baby Wipes Ganti Popok 50s Purple Buy 1 Get 1
matched  train_1241042529 train_2746199767 train_3745037872 train_1667304879 train_932746377
Baju Suster Dokter Anak
matched  train_2105385607 train_1064367871
BUKU TEMATIK SD KELAS 2 TEMA 1 2 3 4 5 6 7 8 SEMESTER 1 DAN 2 KURIKULUM 2013 REVISI KEMENDIKBUD
matched  train_2242533236 train_3106521966 train_3030924187 train_680562184 train_1303632285 train_1245693675 train_2648004517 train_1301516829 train_1619297352 train_2599134160 train_3244767387 train_3046347473 train_1044709849 train_353941535 train_3677421060 train_209193756 train_2103802277 train_346686888 train_3437847332 train_3711182408 train_2103982279 train_1128849795 train_661892807 train_299423300 train_4108661349 train_1869871858 tr

matched  train_285828586 train_3568770156 train_3688161627 train_1594343409 train_3038474963 train_2493407517 train_2910264727 train_2129451181 train_836450777 train_3805633036 train_1822844834 train_63196596 train_2885093405 train_2406956358
[ MINI TRIPOD SPIDER ] Holder U Medium / Tripod Mini Octopus
matched  train_4090097043 train_1252827374 train_563894553 train_3723239604 train_337058529 train_2940867281 train_3650163590 train_623208014 train_1561458167 train_2067217052 train_3759767545 train_2323522436 train_973496278 train_2965165091 train_2938409868 train_2780453609 train_3301603537 train_644504147
[LOGU] Mainan bayi boneka jari animal,boneka main bayi balita, boneka edukasi
matched  train_2962272674 train_3123769162 train_2393141979 train_2328474373
Evergreen Reed Difuser 30ML
matched  train_2437331406
Renu fresh solution 355ml 60ml cairan pembersih softlens TOP no 1
matched  train_130397169 train_1035889078 train_4183230163 train_2074971246 train_2021766358 train_1256708206 t

matched  train_3197183828 train_1748742424 train_3025070727 train_3267286668 train_4068658257 train_6978966 train_1476219261 train_2623691000 train_1507430363 train_4250236185 train_1734979385 train_3621257255 train_3872899411 train_3371636599 train_271260043 train_179227343 train_3174681704
AVAIL DAY USE (BIRU) PROMOOO
matched  train_3838907589 train_4026532747 train_1476219261 train_4109813433 train_3621257255 train_1465256167
[Bayar Ditempat] - Jam Tangan MI 2 Band Watch/ Jam Tangan LED / Jam Tangan Unisex /Jam Sport Digital
matched  train_3241064419 train_3795884238 train_3216356543 train_940886834 train_1373149002 train_2205967347 train_2906961145 train_3491428070 train_2884575572 train_1331848744 train_3520836374
Serba Grosir Murah Bando Elastis Bergelombang Bando Rambut Bentuk Gelombang
matched  train_308261111 train_1935847073
Promo 2019 READY STOCK GOJEK !! Manual Breastpump, Pompa ASI Real Bubee Murah BPA Free
matched  train_1982934939 train_1943466047 train_2880242346 train_

In [65]:
train_df_val

,posting_id,image,image_phash,title,label_group,target,target_string,predicted
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]",train_129225211 train_2278313361,train_129225211 train_2278313361
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",2937985045,"[train_3386243561, train_3423213080]",train_3386243561 train_3423213080,train_3386243561 train_860027362 train_211200223 train_1974987067
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]",train_2288590299 train_3803689425,train_2288590299 train_3803689425
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,4093212188,"[train_2406599165, train_3342059966]",train_2406599165 train_3342059966,train_2406599165 train_3576714541 train_1508100548 train_466135735 train_2043094887 train_1744956981 train_3526771004
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]",train_3369186413 train_921438619,train_3369186413 train_921438619
...,...,...,...,...,...,...,...,...
995,train_3978236021,07df304ea3123879bf0cd0308b93f5d7.jpg,97936a6aa98d6989,7in1 Stick Cup Standing Balon + Tatakan / Tiang Dekorasi Balon,2315452163,"[train_3978236021, train_147687399, train_3338907184, train_1378757570, train_837388645, train_3330791558, train_259228423, train_2227717348, train_112083765, train_1835376406, train_3546013642, train_2756897082, train_3099570705, train_1468848331, train_1320820202]",train_3978236021 train_147687399 train_3338907184 train_1378757570 train_837388645 train_3330791558 train_259228423 train_2227717348 train_112083765 train_1835376406 train_3546013642 train_2756897082 train_3099570705 train_1468848331 train_1320820202,train_3978236021 train_1378757570 train_112083765 train_1835376406 train_3546013642 train_3099570705 train_1320820202
996,train_3755240481,07dfb4dff4bdfb1aa486192f94a8b11e.jpg,b9f0824e945b9b6c,"DRESS ATASAN VANESHA BAHAN SOFTJEANS MURAH, LUCU!!!",2410582194,"[train_3755240481, train_1474163451, train_2297635211, train_3961117946, train_1452043445, train_2072357627]",train_3755240481 train_1474163451 train_2297635211 train_3961117946 train_1452043445 train_2072357627,train_3755240481 train_2884528116 train_3722433776 train_2433779966 train_3961117946 train_3185559167 train_3326482659 train_1892676612
997,train_2711071734,07e130a2369bd30c351a43982d8eae69.jpg,89e1f542325be4e9,Termurah hijab instan anak diamond crepe,1119404634,"[train_1085922914, train_2711071734, train_670048472, train_2460621948, train_3863318592, train_799432869]",train_1085922914 train_2711071734 train_670048472 train_2460621948 train_3863318592 train_799432869,train_2711071734
998,train_2789622683,07e4a963a2f062b8badf1df49abdc585.jpg,96d8a41139e73d27,COD - Easy Touch GCU 3 in 1 / Easytouch Alat test Gula Kolesterol Asam Urat,98262397,"[train_2789622683, train_2622464823]",train_2789622683 train_2622464823,train_2789622683 train_3695725085 train_2622464823 train_4117853354


In [66]:
def match(actual, predicted):
    if actual == predicted:
        return True
    else:
        return False
    
train_df_val['prediction_accuracy'] = train_df_val.apply(lambda x: match(x['target_string'], x['predicted']),axis=1)

<ipython-input-66-0be2f827fa4a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_val['prediction_accuracy'] = train_df_val.apply(lambda x: match(x['target_string'], x['predicted']),axis=1)


In [67]:
prediction_accuracy_v1 = train_df_val.groupby(['prediction_accuracy'])['posting_id'].count()

In [68]:
#train_df_val

In [69]:
#Accuracy
print("Accuracy of V1" , (prediction_accuracy_v1[True]/(prediction_accuracy_v1[True] + prediction_accuracy_v1[False]))*100)

Accuracy of V1 14.499999999999998


In [70]:
#pip install google-cloud-translate==2.0.1

In [71]:
train_df = pd.read_csv('train_val_all_v1.csv')
#train_df = pd.read_csv('../input/shopee-product-matching/train.csv')

In [72]:
#train_df.head(2)

In [73]:
processed_docs = train_df['title'].map(preprocess)
w2v_model = word2vec_model()
w2v_model.save('word2vec_model')

In [74]:
emb_vec = w2v_model.wv

In [75]:
train_df_val = train_df.head(1000)

In [76]:
submission_df_val = pd.DataFrame()
for match_index, match_row in train_df_val.iterrows():
    print(match_row['title'])
    match_title=match_row['title']
    matched = ""
    for index, row in train_df.iterrows():
        current_title=row['title']
        sim = find_similarity(match_title, current_title,emb_vec )
        if sim > 0.9:
            if len(matched)< 1:
                matched = row['posting_id']
            else:
                matched = matched + " " + row['posting_id']
    print("matched ", matched)    
    #train_df_val.loc[match_index]['predicted'] = matched
    train_df_val.loc[match_index,'predicted'] = matched

Paper Bag Victoria Secret


<ipython-input-55-f0148efcbc74>:13: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


matched  train_129225211 train_2278313361
Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE


/Users/sibish/opt/miniconda3/envs/cs7643-a2/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/sibish/opt/miniconda3/envs/cs7643-a2/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


matched  train_3386243561 train_860027362 train_211200223 train_1974987067
Maling TTS Canned Pork Luncheon Meat 397 gr
matched  train_2288590299 train_3803689425
Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi
matched  train_2406599165 train_3576714541 train_2705932532 train_1593362411 train_1508100548 train_466135735 train_2043094887 train_1744956981 train_3526771004
Nescafe \xc3\x89clair Latte 220ml
matched  train_3369186413 train_921438619
CELANA WANITA  (BB 45-84 KG)Harem wanita (bisa cod)
matched  train_2464356923 train_4016842252 train_3717368125 train_605380628 train_1088467991 train_956165758 train_2907999856 train_818562223 train_3201798165 train_718919006 train_3802582694 train_4039731280 train_742888211 train_1915376122 train_997480032 train_2791847582 train_718380210 train_413879832 train_3002955302 train_3395454388 train_2600816306 train_2312840791 train_3648644051 train_1337416323 train_906952721 train_3089842608 train_31

...

matched  train_964138261 train_2097298407 train_4451125 train_631270359 train_2847641686 train_3123142715
PAPYDU ERTOS NIACINAMIDE PLUS WITH ALOEVERA BRIGHTENING MOISTURIZING SERUM 20 ml
matched  train_964138261 train_2097298407 train_528229760 train_826136339 train_4451125 train_631270359 train_2847641686 train_3123142715
Casing Soft Case Oppo A3s A12 A9 A5 2020 A53 A33 2020 A5s A7 Reno4 A12e A1K A37 A71 F7 F9 F9Pro F11 A31 A52 A92 Reno 2F Star Wars Clear Tide Fashion Cover
matched  train_2486804294 train_2027008950 train_3808370752 train_4121268960 train_2614439938 train_3593261092 train_736310903 train_3198752125 train_2398391490 train_586047622 train_92765207 train_1053524389 train_2148139000 train_3198457335 train_1602104591 train_3241085284 train_1081317190 train_3823591564 train_1271316815 train_1684493365 train_2552336621 train_2788619332 train_1128545602 train_126474621 train_1982556945 train_4223515304 train_3821115207 train_3219605857 train_3100671862 train_1692082994 train_

matched  train_1198057711 train_1945506751 train_1131314333 train_374770589 train_2712600415 train_1536577246
ORIGINAL Hardcase GKK CASE 360 Full Protective for Realme 5 / 5s / 5i / 5 Pro
matched  train_4268466355 train_2090194761 train_3251129501
CHUBBY OVERALL / KAFEA OVERALL / LAVETA OVERALL / SNOWY OVERALL REAL PRODUCT BEST SELLER
matched  train_3179426701 train_3191833128 train_1357187258 train_2108922556 train_2809461117 train_95876045 train_3899700010 train_1979989906
Free boneka Sesuai Foto Tas Ransel Weedo Love Tas Murah Tas Batam
matched  train_3064668012
ANTICRACK / ANTI CRACK CASE SONY Xperia XZs XZ XZ1 5 1 10 XA2 XA1 PLUS L3 XZ3 XZ2 Compact Ultra L2
matched  train_2309760239
Momogi Jagung Bakar / Cokelat / Cappucino / Tutti Frutti 20 pcs x 8 gr Cemilan Stik Keju MURAH 1 DUS
matched  train_925751208 train_4179871082 train_3319173288 train_776210967 train_1187596622
Vidoran Xmart 3+ 1000gr Madu, Vanilla / Vidoran 3 Plus 1kg Usia 3-5 Tahun
matched  train_128219791 train_26028

matched  train_3728233037
Extra Bubble Wrap dan Dus Packing
matched  train_3284960524 train_443869273 train_4226152332 train_1437764574 train_584097694 train_1752751945 train_1010043803 train_3056813730 train_3370528070 train_1466235757 train_488563414 train_3429918070 train_382020659 train_1454497638 train_2514122916 train_887858535 train_2832484646 train_2626565979 train_1067451590 train_276361825 train_2498383951 train_418910179 train_989784973 train_3688770328 train_1652102979 train_1429501207
Cream HN Ori 15gr/krim HN ori,beli 10 gratis 1
matched  train_3383696066 train_1185643501 train_2746419015 train_96842855 train_548039175
Ovomaltine Crunchy Cream 240gr
matched  train_482015545 train_3329475700 train_680463922 train_2998627649 train_19562926 train_3597993320 train_331310124 train_3919541698
SAHOO alat reparasi tambal ban sepeda bicycle repair toolkit
matched  train_192154327 train_1686054869
Tutup Pentil Nitex Mdl Bikers Full Cnc Beat Mio Aerox Nmax 2020 Xmax Scoopy Xeon PCX 

matched  train_1164271328 train_2355485408 train_2556781752 train_2515193197 train_419246294 train_2640918643 train_3071708946 train_3144366839 train_1210040144 train_2387753807 train_2986446502 train_1309289735 train_2072165327 train_1361510065 train_2550577184 train_4102163551 train_236443778 train_3293716060 train_3680816797 train_200649983 train_3303300755 train_2013267906 train_1120925791 train_452298409 train_1319025859 train_396196244 train_702603526 train_3176906750 train_2346721309
Pompa Injak merk BESTWAY 62007
matched  train_1455917465 train_1390886618 train_379528076 train_2708745638 train_3474797264
Sunsilk Shampo Black Shine 680 ml - Sampo Rambut Sampo Salon Shampo Rambut Berminyak Shampoo Salon
matched  train_1988357862
8 Bola Bantal Pijat portable Car and Home Masaage Pillow 202-01
matched  train_3223029042 train_1870943913 train_3630367129 train_916885828 train_3725815927 train_320344199 train_531287731 train_2475193119 train_1890569721 train_3459133705 train_116443095

matched  train_34881320 train_3903462741 train_935224973 train_2263502148 train_2854784767 train_2600864012 train_862364804 train_619297071 train_3837607576 train_2071406463 train_3880383403 train_3544770416 train_3832103819 train_2040317333 train_3270632649 train_2905875577 train_1796348010 train_984464091 train_1683564540 train_128664522 train_3823100252 train_2404106367 train_143166491 train_3901129094 train_1019174187 train_2748381050 train_2829334594 train_2850264987 train_1508335846 train_3905397940 train_1011736295 train_2949480753 train_4080544101 train_1344648324 train_3804392948 train_2299104562 train_2470222196 train_3601127643 train_209530549 train_1888464306 train_3514865662 train_396036486 train_3771232329 train_4151900908 train_1608278450 train_2294053571 train_4223583444 train_1773197852 train_381554002 train_3950993681 train_104288742 train_3239425368 train_1516573836 train_1061380742 train_708708805 train_306951335 train_593805901
Tote bag wanita LUCK FOR YOU / Tote b

In [77]:
def match(actual, predicted):
    if actual == predicted:
        return True
    else:
        return False
    
train_df_val['prediction_accuracy'] = train_df_val.apply(lambda x: match(x['target_string'], x['predicted']),axis=1)

<ipython-input-77-0be2f827fa4a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_val['prediction_accuracy'] = train_df_val.apply(lambda x: match(x['target_string'], x['predicted']),axis=1)


In [78]:
prediction_accuracy_v1 = train_df_val.groupby(['prediction_accuracy'])['posting_id'].count()

In [79]:
#Accuracy
print("Accuracy of V2" , (prediction_accuracy_v1[True]/(prediction_accuracy_v1[True] + prediction_accuracy_v1[False]))*100)

Accuracy of V2 14.6
